## TODO: 
- Add Markdown for each Tool function
- Review if we Reviewer can reuse Researcher tools to fetch Ticker data
- Add comments to Reviewer Agent and Review Task
- 

---

# AAI-520-02 Final Team Project: Multi-Agent System

#### Contributors:
- Alexander J Padin
- Thomas Geraci
- Ali Mohtat

## Imports

In [5]:
import os
from typing import Dict, Any, List
from dotenv import load_dotenv
import pandas as pd
import yfinance as yf
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool 
from typing import Dict, Any
from IPython.display import display, Markdown
from pathlib import Path

load_dotenv() # load .env variable with API Keys (see .env-example)

DATA_DIR = Path("data"); DATA_DIR.mkdir(exist_ok=True)
OUT_DIR  = Path("outputs"); OUT_DIR.mkdir(exist_ok=True)
MEM_DIR  = Path("memory"); MEM_DIR.mkdir(exist_ok=True)

## Large Language Models (LLMs)  
In our system, LLMs are the foundational models that power each agent's reasoning, text generation, and decision-making. We explicitly define which LLM each agent will use (e.g. OpenAI’s GPT models or Anthropic’s Claude) so the agents operate with predictable performance and behavior.

In [7]:
claude_llm = LLM(
    model="claude-3-5-sonnet-20240620",
    base_url="https://api.anthropic.com",
    api_key=os.environ["ANTHROPIC_API_KEY"]
)

openai_llm = LLM(
    model="openai/gpt-4o",
    api_key=os.environ["OPENAI_API_KEY"]
)

## Helper Functions

In [9]:
#Knowledge Base helpers
def _kb_path(ticker: str) -> Path:
    p = MEM_DIR / f"kb_{ticker.upper()}.jsonl"
    p.touch(exist_ok=True)
    return p

def kb_append_lesson(ticker: str, lesson: dict) -> None:
    p = _kb_path(ticker)
    with p.open("a", encoding="utf-8") as f:
        f.write(json.dumps(lesson, ensure_ascii=False) + "\n")

def kb_load_lessons(ticker: str) -> list[dict]:
    p = _kb_path(ticker)
    out = []
    with p.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                out.append(json.loads(line))
    return out

## Tools
Tools are modular, executable functions that agents can call to perform tasks (e.g., fetching data or interacting with APIs). In CrewAI, we wrap a function with `@tool("Tool Name")`, which registers it with metadata. The agent can then decide when to call which tool and how to use its output.  

### Tools in this project  
- `yf_prices(ticker, period='90d', interval='1d')`: fetch recent OHLCV (Open-High-Low-Close-Volume) price data  
- `yf_fundamentals(ticker)`: retrieve a compact fundamentals snapshot (e.g. market cap, P/E, EPS)  
- `yf_dividends(ticker, limit=5)`: fetch recent dividend events (up to N)  
- `yf_calendar(ticker)`: get upcoming or recent calendar events (e.g. earnings)  

### OHLCV Fetcher
This tool fetches recent OHLCV (Open, High, Low, Close, Volume) data for a given stock ticker over a specified time period (90 days) using `yfinance`. It returns up to the last 10 rows of data (the “tail”) plus metadata indicating how many rows were available. If no data is found, it returns an empty result with zero rows.

In [12]:
@tool("Fetch recent OHLCV prices from Yahoo Finance")
def yf_prices(ticker: str, period: str = "90d", interval: str = "1d"):
    """Returns recent OHLCV prices from Yahoo Finance""" # need docstring for tools
    
    tk = yf.Ticker(ticker)
    df = tk.history(period=period, interval=interval, auto_adjust=False)
    
    if df is None or df.empty: # if empty
        return {"ticker": ticker, "period": period, "interval": interval, "rows": 0, "tail10": []}
        
    out = df.reset_index().tail(10).to_dict(orient="records")
    return {"ticker": ticker, "period": period, "interval": interval, "rows": len(df), "tail10": out}

### Fundamentals Snapshot Fetcher
This tool fetches a compact fundamentals snapshot for a given ticker using `yfinance`. It tries to use `fast_info` (a lightweight subset of metadata) when available; if not, it falls back to `Ticker.info`. The returned dict includes fields like last price, market cap, P/E, EPS, 52-week high/low, and others if available.

In [14]:
@tool("Fetch a fundamentals snapshot from Yahoo Finance")
def yf_fundamentals(ticker: str) -> Dict[str, Any]:
    """Returns a fundamentals snapshot from Yahoo Finance"""
    
    tk = yf.Ticker(ticker)
    info = getattr(tk, "fast_info", None)
    
    if info:
        # if fast_info exists, use selected attributes
        attrs = [
            "last_price", "market_cap", "currency", 
            "year_high", "year_low", "shares_outstanding"
        ]
        fi = {k: getattr(info, k, None) for k in attrs}
        
    else: # else, get data form Ticker.info
        try:
            raw = tk.info
        except Exception:
            raw = {}
        fi = {
            "last_price": raw.get("currentPrice"),
            "market_cap": raw.get("marketCap"),
            "currency": raw.get("currency"),
            "year_high": raw.get("fiftyTwoWeekHigh"),
            "year_low": raw.get("fiftyTwoWeekLow"),
            "shares_outstanding": raw.get("sharesOutstanding"),
            "trailing_pe": raw.get("trailingPE"),
            "forward_pe": raw.get("forwardPE"),
            "eps": raw.get("trailingEps") or raw.get("epsTrailingTwelveMonths"),
            "short_name": raw.get("shortName"),
            "beta": raw.get("beta"),
        }
        
    return {"ticker": ticker, "fundamentals": fi}

### Dividends History Fetcher
This tool retrieves the most recent dividend events for a given ticker via `yfinance`. It returns up to a given number of past dividends (5), each with a date and dividend amount. If the ticker has no dividends or the data is empty, it returns an empty list.

In [16]:
@tool("Fetch dividends history (compact)")
def yf_dividends(ticker: str, limit: int = 5) -> Dict[str, Any]:
    """Returns dividends history (compact)"""
    
    tk = yf.Ticker(ticker)
    s = tk.dividends
    
    if s is None or s.empty: # if no dividends 
        return {"ticker": ticker, "dividends": []}
        
    df = s.reset_index().rename(columns={"Date": "date", "Dividends": "dividend"})
    return {"ticker": ticker, "dividends": df.tail(limit).to_dict(orient="records")}

### Earnings Calendar & Next Events Fetcher
This tool attempts to fetch upcoming or recent calendar events (e.g. earnings, dividends) for a ticker via `yfinance.Ticker.calendar`. It returns a structured list of event names and values if available; otherwise it returns an empty list.

In [18]:
@tool("Fetch earnings calendar / next events")
def yf_calendar(ticker: str) -> Dict[str, Any]:
    """Returns earnings calendar / next events"""
    
    tk = yf.Ticker(ticker)
    
    try:
        cal = tk.calendar
        if cal is None or cal.empty: # if no events
            return {"ticker": ticker, "calendar": []}
            
        df = cal.reset_index().rename(columns={"index": "event", 0: "value"})
        return {"ticker": ticker, "calendar": df.to_dict(orient="records")}
    except Exception: # if no success with yfinance endpoint
        return {"ticker": ticker, "calendar": []}

---

# TODO: Review tools

In [21]:
@tool("Return past one-sentence takeaways (lessons) for a ticker, one per line.")
def load_kb_lessons_tool(ticker: str) -> str:
    """Return past one-sentence takeaways (lessons) for a ticker, one per line."""
    lessons = kb_load_lessons(ticker)
    if not lessons:
        return "(no prior lessons)"
    return "\n".join(f"- {i+1}. {row.get('lesson', '')}" for i, row in enumerate(lessons))


In [22]:

@tool("Append a new one-sentence lesson for a ticker to the KB.")
def save_kb_lesson_tool(ticker: str, lesson: str) -> str:
    """Append a new one-sentence lesson for a ticker to the KB."""
    kb_append_lesson(ticker, {"lesson": lesson, "ts": time.time()})
    return f"Saved lesson for {ticker}: {lesson}"

In [23]:

@tool("Fetch OHLCV with yfinance and return a compact summary string.")
def yf_download_tool(ticker: str, period: str = "6mo", interval: str = "1d") -> str:
    """Fetch OHLCV with yfinance and return a compact summary string."""
    df = yf.download(
        ticker, period=period, interval=interval,
        progress=False, group_by="column", auto_adjust=True
    )
    if df.empty:
        return f"{ticker} {period}/{interval}: no data"

    if hasattr(df.columns, "levels") and len(getattr(df.columns, "levels", [])) > 1:
        df.columns = df.columns.get_level_values(0)

    try:
        avg_close = float(df["Close"].mean())
    except Exception:
        avg_close = float(df.filter(regex="(?i)close").mean(numeric_only=True).iloc[0])

    try:
        avg_vol = float(df["Volume"].mean())
    except Exception:
        avg_vol = float(df.filter(regex="(?i)volume").mean(numeric_only=True).iloc[0])

    return (
        f"{ticker} {period}/{interval}: rows={len(df)}, "
        f"avgClose={avg_close:.2f}, avgVol={avg_vol:.0f}"
    )


In [24]:
@tool("Return a terse snapshot of recent trend for Close price.")
def series_snapshot_tool(ticker: str, period: str = "1y", interval: str = "1d") -> str:
    """Return a terse snapshot of recent trend for Close price."""
    df = yf.download(
        ticker, period=period, interval=interval,
        progress=False, group_by="column", auto_adjust=True
    )
    if df.empty:
        return f"{ticker} trend: no data"

    if hasattr(df.columns, "levels") and len(getattr(df.columns, "levels", [])) > 1:
        df.columns = df.columns.get_level_values(0)

    closes = df["Close"].astype(float)
    ret = (closes.iloc[-1] / closes.iloc[0]) - 1.0
    return f"{ticker} trend over {period} @ {interval}: {ret:+.1%}"

---

## Agents
An agent is an autonomous AI entity defined by a role, goal, a backstory (context) and tools. It reasons, plans, and acts on tasks using its tools, and can collaborate within a crew.  

### Agents in the Crew:
- Planner: Designs a structured, step-by-step plan (5–7 bullets) for how to research a given ticker using the Yahoo Finance tools.
- Researcher: Executes the plan by calling the Yahoo Finance tools (prices, fundamentals, dividends, calendar) and synthesizes findings, including a Buy/Sell/Hold recommendation and compact data summary.
- Reviewer: Reviews the Researcher's draft output and generates feedback and improvement suggestions (a review plan) for the next run.
- Optimizer: Takes the Researcher's work plus Reviewer feedback; modifies and refines the draft accordingly, and stores lessons learned to improve future runs.

### Planner Agent
This agent's role is to design a compact, tactical research plan for a given ticker. It does not fetch data itself, instead it creates a 5-7 step sequence that a Researcher agent can follow using only the Yahoo Finance tools (prices, fundamentals, dividends, calendar).  

In [28]:
planner = Agent(
    role="Planner", # role
    goal=( # goal prompt
        "Design a concise, step-by-step plan (5–7 bullets) for researching {ticker} "
        "using only the Yahoo Finance tools available to the Researcher."
    ),
    backstory=( # context
        "You are a methodical planning specialist. Your job is to outline an efficient "
        "sequence of steps that the Researcher can follow to gather price context, "
        "key fundamentals, dividends, upcoming events, and any red-flag checks."
    ),
    tools=[], # no tools used
    allow_delegation=False, # no delegation to other agents
    memory=False, # no retain of memory (maybe?)
    verbose=True, # enable logging of internal reasoning
    llm=openai_llm, # the LLM to be used; gpt-4o-mini is too small for this task
    max_iter=2, # number of reasoning iterations
    max_rpm=30, # max request per mins
)

### Researcher Agent
This agent uses Yahoo Finance tools (prices, fundamentals, dividends, calendar) to generate a compact, data-driven analysis of a given stock. Its output includes a ~200-word synthesis, a Buy/Sell/Hold recommendation, and a concise data snapshot.  

In [30]:
researcher = Agent(
    role="Researcher",
    goal=(
        "Use the provided Yahoo Finance tools to gather a concise, data-backed snapshot "
        "for a given stock ticker. Produce a ~200-word synthesis, a Buy/Sell/Hold recommendation, "
        "and a compact data summary."
    ),
    backstory=(
        "You are a pragmatic equity researcher. You rely only on Yahoo Finance via the tools provided "
        "and call out any missing or uncertain data. Keep outputs crisp and actionable."
    ),
    tools=[yf_prices, yf_fundamentals, yf_dividends, yf_calendar],
    allow_delegation=False,
    memory=False,
    verbose=True,
    llm=openai_llm,
    max_iter=3,
    max_rpm=30,
)

### Reviewer Agent

This is a quality-assurance agent that rigorously critiques the Researcher's draft. It assigns a grade (1–5), provides a concise rationale, and produces a concrete improvement plan (3–6 bullets). It also captures a single lesson sentence for future use.  

In [32]:
#Agent
reviewer = Agent(
    role="Reviewer",
    goal=(
        "Judge the Researcher’s draft strictly, assign a single 1–5 rank, "
        "provide rationale, and produce a concrete improvement plan for the Optimizer. "
        "Store a concise lesson."
    ),
    backstory=(
        "A meticulous investment research QA who grades with a tough rubric "
        "and writes crisp, actionable feedback."
    ),
    tools=[load_kb_lessons_tool, save_kb_lesson_tool, yf_download_tool, series_snapshot_tool],
    allow_delegation=False,
    verbose=True,
    llm=claude_llm,
)

## Tasks
A Task is an assignment that an Agent must complete. It contains everything needed to perform such as the description telling the agent what to do, the agent responsible for executing it, expected outputs/format, context and tools that the agent can use during task execution. 

### Planning Task 
This  Task directs the Planner Agent to produce a structured 5–7 step **RESEARCH_PLAN** for the Researcher agent, using only the allowed Yahoo Finance tools (`yf_prices`, `yf_fundamentals`, `yf_dividends`, `yf_calendar`).  

In [35]:
PLANNING_PROMPT = """You are the Equity Research Planner.

Ticker: {ticker}

INSTRUCTIONS:
- Draft a 5–7 step RESEARCH_PLAN for the Researcher that uses ONLY these tools:
  - yf_prices(ticker, period='90d', interval='1d')
  - yf_fundamentals(ticker)
  - yf_dividends(ticker, limit=5)
  - yf_calendar(ticker)
- Order steps logically (price context → fundamentals → dividends → calendar → red flags).
- Be specific about periods/parameters the Researcher should try.
- If data could be missing, note a fallback (e.g., try a longer period).

OUTPUT:
**RESEARCH_PLAN** — numbered bullets
"""

# creates a CrewAI task
plan_research = Task(
    description=PLANNING_PROMPT, # prompt with the instructions for the task
    agent=planner, # the agent responsable for this task (Planner)
    expected_output="RESEARCH_PLAN" # the expected output
)

### Research Task: 
This Task directs the Researcher to produce a data-driven SYNTHESIS, a Buy/Sell/Hold recommendation with rationale, and a compact DATA SNAPSHOT. It instructs the agent to consume a prior RESEARCH_PLAN or draft one itself, then use the Yahoo Finance tools (`yf_prices`, `yf_fundamentals`, `yf_dividends`, `yf_calendar`) to pull data and structure the output accordingly.

In [37]:
RESEARCH_PROMPT = """You are the Equity Researcher.

Ticker: {ticker}

TOOLS AVAILABLE:
- yf_prices(ticker, period='90d', interval='1d'): recent OHLCV
- yf_fundamentals(ticker): compact fundamentals snapshot
- yf_dividends(ticker, limit=5): recent dividends
- yf_calendar(ticker): next/recent calendar entries (e.g., earnings)

INSTRUCTIONS:
0) Follow the RESEARCH_PLAN provided in context. If no plan is present, quickly draft a brief one before proceeding.
1) Call the tools you need to understand {ticker}'s recent behavior and snapshot.
2) Summarize price context, key fundamentals (best-effort), any dividends, and upcoming events if present.
3) If a tool returns empty/missing fields, acknowledge it and suggest a reasonable fallback (e.g., longer period).
4) Produce a Buy/Sell/Hold recommendation.

OUTPUT:
**SYNTHESIS (~150–250 words)**
**RECOMMENDATION** — a one-word classification (BUY/SELL/HOLD) plus one short sentence of rationale, and include "recommendation" inside the JSON snapshot.
**DATA SNAPSHOT** — compact JSON
"""

research_ticker = Task(
    description=RESEARCH_PROMPT,
    agent=researcher, # executed by Researcher Agent
    expected_output="SYNTHESIS, RECOMMENDATION, DATA SNAPSHOT",
    context=[plan_research] # passes the Planner's output to the Researcher as context
)

### Review Task: 
This task takes the Researcher’s draft output (via `context=[research_ticker]`) and critiques it.  

The Reviewer:
- Assigns a grade (1–5),
- Writes a rationale** (2–5 sentences),
- Crafts a concrete improvement plan (3–6 bullets),
- Produces a single lesson sentence to store for future runs.

The Reviewer task runs after the Researcher in the sequential Crew, ensuring it always has the Researcher’s output to review.


In [39]:
REVIEW_PROMPT = """You are reviewing an investment research draft.

Inputs:
- Ticker: {ticker}

Context (tools available):
- You may load prior one-sentence takeaways (lessons) for this ticker.
- You may fetch a quick price/volume summary and a simple trend snapshot.

Your job:
1) Assign a grade (integer, 1–5) for overall quality (correctness, completeness, clarity).
2) Write a short rationale (2–5 crisp sentences).
3) Give a concrete improvement plan (3–6 bullet steps).
4) Write exactly one sentence 'lesson' (to save for future reviewers).

Output: STRICT JSON with keys:
{{
  "grade": 1-5 integer,
  "rationale": "string",
  "plan": ["step 1", "step 2", ...],
  "lesson": "one sentence"
}}

Be strict and specific. No text outside the JSON.
"""

review_task = Task(
    description=REVIEW_PROMPT,
    agent=reviewer,
    expected_output='Strict JSON with keys grade, rationale, plan, lesson.',
    output_file=str(OUT_DIR / "reviewer_output.json"),
    context=[research_ticker]  # added Researcher's task output
)

## Crew Setup
Here we create a Crew that coordinates the agents and the tasks.

When `crew.kickoff()` is called, the Planner will produce a plan, then the Researcher uses that plan to drive its analysis....  


In [41]:
crew = Crew(
    agents=[planner, researcher, reviewer], # list agents        
    tasks=[plan_research, research_ticker, review_task], #  list of tasks to execute in order
    process=Process.sequential, # task to run one after another
    verbose=True # enable logging
)

crew

Crew(id=7d98aad5-2ab9-429c-a70b-ae1f4b8d68ef, process=Process.sequential, number_of_agents=3, number_of_tasks=3)

In [42]:
# execute the crew's tasks in sequence using an input
result = crew.kickoff(inputs={"ticker": "AAPL"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7d98aad5-2ab9-429c-a70b-ae1f4b8d68ef                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Planner                                                                                                 │
│                                                                                                                 │
│  Task: You are the Equity Research Planner.                                                                     │
│                                                                                                                 │
│  Ticker: AAPL                                                                                                   │
│                                                                                                                 │
│  INSTRUCTIONS:                                                                                                  │
│  - Draft a 5–7 step RESEARCH_PLAN for the Researcher that uses ONLY these tools:                                │
│    - yf_prices(ticker, period='90d', interval='1d')                                                             │
│    - yf_fundamentals(ticker)                                                                                    │
│    - yf_dividends(ticker, limit=5)                                                                              │
│    - yf_calendar(ticker)                                                                                        │
│  - Order steps logically (price context → fundamentals → dividends → calendar → red flags).                     │
│  - Be specific about periods/parameters the Researcher should try.                                              │
│  - If data could be missing, note a fallback (e.g., try a longer period).                                       │
│                                                                                                                 │
│  OUTPUT:                                                                                                        │
│  **RESEARCH_PLAN** — numbered bullets                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Planner                                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **RESEARCH_PLAN**                                                                                              │
│                                                                                                                 │
│  1. **Price Context**: Get the recent price data for AAPL by using the `yf_prices(ticker, period='90d',         │
│  interval='1d')` function. This will give you daily closing prices for the past 90 days to analyze the stock's  │
│  recent price movements and volatility.                                                                         │
│                                                                                                                 │
│  2. **Key Fundamentals**: Use the `yf_fundamentals(ticker)` function to retrieve the key financial metrics and  │
│  ratios for AAPL. This includes examining aspects such as P/E ratio, revenue, earnings, and any significant     │
│  trends in the company's financial health.                                                                      │
│                                                                                                                 │
│  3. **Dividends Information**: Check the latest dividend data for AAPL using `yf_dividends(ticker, limit=5)`.   │
│  This will provide information on the most recent dividend payments and history, helping assess income          │
│  potential for investors.                                                                                       │
│                                                                                                                 │
│  4. **Upcoming Events**: Access the upcoming corporate events like earnings announcements, shareholder          │
│  meetings, etc., with `yf_calendar(ticker)`. This helps in preparing for events that might impact the stock     │
│  price or provide significant information.                                                                      │
│                                                                                                                 │
│  5. **Red Flag Checks**: Review all collected data for any significant concerns or warnings. Look for any       │
│  anomalies in the fundamentals, sudden changes in dividends, or crucial dates in the calendar that could pose   │
│  risks, such as upcoming earnings reports during volatile market conditions.                                    │
│                                                                                                                 │
│  By following these steps, the Researcher will systematically gather and analyze all essential data on AAPL,    │
│  forming a comprehensive overview necessary for informed decision-making.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 10bca176-1de4-4306-83c9-c1edc7f9a0a5                                                                     │
│  Agent: Planner                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  (no prior lessons)                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Reviewer                                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "grade": 4,                                                                                                  │
│    "rationale": "The research provides a solid overview of AAPL's recent performance and financial position.    │
│  It accurately reflects the stock's recent upward trend and provides relevant financial metrics. The            │
│  recommendation to HOLD is justified given the current market conditions and AAPL's stable performance.         │
│  However, the analysis could be more forward-looking and provide deeper insights into potential catalysts or    │
│  risks.",                                                                                                       │
│    "plan": [                                                                                                    │
│      "Incorporate more forward-looking analysis, including potential catalysts or risks for AAPL",              │
│      "Add comparison to sector peers or broader market indices for context",                                    │
│      "Include more specific financial metrics such as P/E ratio or revenue growth",                             │
│      "Analyze the impact of recent product launches or company announcements",                                  │
│      "Consider the effects of broader economic factors on AAPL's performance"                                   │
│    ],                                                                                                           │
│    "lesson": "AAPL research should balance current performance with forward-looking analysis and broader        │
│  market context to provide comprehensive investment insights."                                                  │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 38f63a17-4e3b-441e-8eff-0eee159cd7ce                                                                     │
│  Agent: Reviewer                                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7d98aad5-2ab9-429c-a70b-ae1f4b8d68ef                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "grade": 4,                                                                                                  │
│    "rationale": "The research provides a solid overview of AAPL's recent performance and financial position.    │
│  It accurately reflects the stock's recent upward trend and provides relevant financial metrics. The            │
│  recommendation to HOLD is justified given the current market conditions and AAPL's stable performance.         │
│  However, the analysis could be more forward-looking and provide deeper insights into potential catalysts or    │
│  risks.",                                                                                                       │
│    "plan": [                                                                                                    │
│      "Incorporate more forward-looking analysis, including potential catalysts or risks for AAPL",              │
│      "Add comparison to sector peers or broader market indices for context",                                    │
│      "Include more specific financial metrics such as P/E ratio or revenue growth",                             │
│      "Analyze the impact of recent product launches or company announcements",                                  │
│      "Consider the effects of broader economic factors on AAPL's performance"                                   │
│    ],                                                                                                           │
│    "lesson": "AAPL research should balance current performance with forward-looking analysis and broader        │
│  market context to provide comprehensive investment insights."                                                  │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [43]:
# print each task result 
display(Markdown("# Planner Agent Output"))
display(Markdown(plan_research.output.raw))

# Planner Agent Output

**RESEARCH_PLAN**

1. **Price Context**: Get the recent price data for AAPL by using the `yf_prices(ticker, period='90d', interval='1d')` function. This will give you daily closing prices for the past 90 days to analyze the stock's recent price movements and volatility.

2. **Key Fundamentals**: Use the `yf_fundamentals(ticker)` function to retrieve the key financial metrics and ratios for AAPL. This includes examining aspects such as P/E ratio, revenue, earnings, and any significant trends in the company's financial health.

3. **Dividends Information**: Check the latest dividend data for AAPL using `yf_dividends(ticker, limit=5)`. This will provide information on the most recent dividend payments and history, helping assess income potential for investors.

4. **Upcoming Events**: Access the upcoming corporate events like earnings announcements, shareholder meetings, etc., with `yf_calendar(ticker)`. This helps in preparing for events that might impact the stock price or provide significant information.

5. **Red Flag Checks**: Review all collected data for any significant concerns or warnings. Look for any anomalies in the fundamentals, sudden changes in dividends, or crucial dates in the calendar that could pose risks, such as upcoming earnings reports during volatile market conditions.

By following these steps, the Researcher will systematically gather and analyze all essential data on AAPL, forming a comprehensive overview necessary for informed decision-making.

In [44]:
display(Markdown("# Researcher Agent Output"))
display(Markdown(research_ticker.output.raw))

# Researcher Agent Output

**SYNTHESIS**
Apple Inc. (AAPL) has shown some volatility over the recent 90-day period, with the price moving between $248.12 to $259.24. The stock closed at $258.02 as of the last trading session, indicating a recent upward trend from its lower 90-day levels. From a fundamentals perspective, AAPL boasts a significant market capitalization of approximately $3.83 trillion and remains strong with a 52-week high of $260.10 and a low of $169.21. AAPL's consistency in dividend payouts is evident with its regular quarterly dividends, the latest being $0.26, which suggests stable income potential for investors. While no specific upcoming events were reported in the earnings calendar, investors should stay vigilant for any forthcoming announcements that could impact stock performance.

**RECOMMENDATION**: HOLD — AAPL remains a well-capitalized and fundamentally strong company, though its recent price movement does not present a compelling buy or sell trigger. Maintaining current positions could be beneficial as the market trends are closely monitored.

**DATA SNAPSHOT**
```json
{
  "ticker": "AAPL",
  "last_price": 258.02,
  "90_day_range": {"low": 248.12, "high": 259.24},
  "market_cap": 3829117264758.606,
  "year_high": 260.10,
  "year_low": 169.21,
  "recent_dividends": [{"date": "2025-05-12", "dividend": 0.26}, {"date": "2025-08-11", "dividend": 0.26}],
  "upcoming_events": [],
  "recommendation": "HOLD"
}
```

In [45]:
display(Markdown("# Reviewer Agent Output"))
display(Markdown(review_task.output.raw))

# Reviewer Agent Output

{
  "grade": 4,
  "rationale": "The research provides a solid overview of AAPL's recent performance and financial position. It accurately reflects the stock's recent upward trend and provides relevant financial metrics. The recommendation to HOLD is justified given the current market conditions and AAPL's stable performance. However, the analysis could be more forward-looking and provide deeper insights into potential catalysts or risks.",
  "plan": [
    "Incorporate more forward-looking analysis, including potential catalysts or risks for AAPL",
    "Add comparison to sector peers or broader market indices for context",
    "Include more specific financial metrics such as P/E ratio or revenue growth",
    "Analyze the impact of recent product launches or company announcements",
    "Consider the effects of broader economic factors on AAPL's performance"
  ],
  "lesson": "AAPL research should balance current performance with forward-looking analysis and broader market context to provide comprehensive investment insights."
}